In [ ]:
import pickle
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from diffusers import StableDiffusionPipeline
from utils.dataset_loader import CustomDatasetWithLatent

In [ ]:
dataset_name = "../latent_files/dataset_with_latents_t2i.pkl"
model_name = "../trained_models/histopathology-diffusion-t2i-256"
device = "cuda:2"
dataset = pickle.load(open(dataset_name, "rb"))
dataset = CustomDatasetWithLatent(dataset)
data_loader = DataLoader(dataset, batch_size=64, shuffle=False)

In [ ]:
pipeline = StableDiffusionPipeline.from_pretrained(model_name, safety_checker=None)
pipeline = pipeline.to(device)

In [ ]:
outs = []
for data in data_loader:
    embeddings = data["embedding"]
    embeddings = embeddings#.unsqueeze(1)
    output = pipeline(
        prompt_embeds=embeddings,
        guidance_scale=0.0,
        num_inference_steps=40
    ).images

    for i in range(len(output)):
        outs.append({
            'generated': output[i]
        })
    
    if len(outs) >= 40:
        break

In [ ]:
print(len(outs))

In [ ]:
display(outs[30]['generated'])

In [ ]:
""" with open('generated_images_i2i.pkl', 'wb') as f:
    pickle.dump(outs, f) """

In [ ]:
selected_images = 40
fig, axs = plt.subplots(4, 10, figsize=(20, 8))
fig.suptitle('Text to Image Generation')

for i in range(selected_images):
    generated = outs[i]['generated']
    axs[i // 10, i % 10].imshow(generated)
    axs[i // 10, i % 10].axis('off')

plt.tight_layout()
plt.show()
fig.savefig('generated_images_t2i.png')